In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime
import textwrap
import random

2025-10-29 07:00:34.147646: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 07:00:34.225873: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 07:00:38.592452: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
RANDOM_STATE = 42
path_DATA = "../../../data"

# Dataset path
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"
zero_shot_path = f"{path_DATA}/zero_outputs"

# Zero-shot file path
path_df_zero = f"{zero_shot_path}/zero_shot_test_results.csv"

# TF-IDF file path
tfidf_path = f"{zero_shot_path}/tfidf/test_predictions.csv"

# Prompting path
prompting_path = f"{zero_shot_path}/llama_8B_results_test.csv"



TESTING = False

if TESTING:
    NROWS = 50
else:
    NROWS = None
    


In [18]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None):
    
    df = pd.read_csv(csv_path, nrows=sample_size)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)

    df['original_index'] = df.index # Indices correctos despues de la eliminación
    X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    y = df['Explicit_binary']
    display(df)
    return X, y, df['original_index'], df['song'], df['Artist(s)'] 

In [19]:
# Load dataset
# df_song = pd.read_csv(csv_path, nrows=NROWS)
df_song,y, _, song_names, artists = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS)
display(df_song.head())

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3,song_normalized,artist_normalized,original_index,Explicit_binary
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,sadness,hip hop,Thr!!!er,29th April 2013,D min,105,...,Toby Keith,Drinks After Work,0.983719,Space,Neighbourhood,0.983236,even when the waters cold,!!!,0,0
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,sadness,hip hop,Thr!!!er,29th April 2013,A# min,117,...,When In Rome,Heaven Knows,0.990905,Justice Crew,Everybody,0.984483,one girl one boy,!!!,1,0
2,!zeesh,Camp Fire,[Verse 1: Bill] Yeah You don't got bars that d...,01:52,anger,hip hop,Campfire (Freestyle),25th April 2023,D Maj,101,...,Knucks,From Rome With Love,0.947579,"FILV,Edmofo,Emma Peters,Jvstin",All the Good Girls Go To Hell,0.940189,camp fire,!zeesh,2,1
3,"""Weird Al"" Yankovic",Amish Paradise,[Verse 1] As I walk through the valley where I...,03:22,joy,"rock,pop,comedy",Bad Hair Day,31st December 1996,G# Maj,81,...,"""Weird Al"" Yankovic",George Of The Jungle,0.999424,Danity Kane,Showstopper Remix,0.980017,amish paradise,"""weird al"" yankovic",3,0
4,"""Weird Al"" Yankovic",Word Crimes,[Intro] Everybody shut up! (Woo!) Everyone lis...,03:43,joy,"rock,pop,comedy",Mandatory Fun,15th July 2014,G Maj,122,...,Summer Thieves,Bottom of Your Face,0.989532,Summer Thieves,NFTs,0.989532,word crimes,"""weird al"" yankovic",4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108120,落日飛車 Sunset Rollercoaster,I Know You Know I Love You,"Watched the sky, you know I Like a star shinin...",02:00,joy,hip hop,BOSSA NOVA,22nd September 2011,E Maj,144,...,"Bing Crosby,Buddy Cole Trio",For Me And My Gal - With Judy Garland,0.989260,Lilla Crawford,I Know Things Now,0.986019,i know you know i love you,落日飛車 sunset rollercoaster,108120,0
108121,落日飛車 Sunset Rollercoaster,Candlelight,[Verse 1] There came the storm that night It b...,03:21,sadness,hip hop,Spotify Singles,21st July 2023,G min,141,...,Zach Williams,The Struggle,0.988890,Ocie Elliott,Rainbow and Arrow,0.988112,candlelight,落日飛車 sunset rollercoaster,108121,0
108122,落日飛車 Sunset Rollercoaster,Let There Be Light Again,[Verse 1] Let there be light again Time to see...,03:47,sadness,hip hop,Let There Be Light Again,7th September 2021,C# min,84,...,Yo La Tengo,Our Way to Fall,0.968445,Brian Eno,Sky Saw,0.968329,let there be light again,落日飛車 sunset rollercoaster,108122,0
108123,郭顶,Perfect,[Hook] I was never perfect when I had you But ...,05:25,sadness,pop,飞行器的执行周期,25th November 2016,F Maj,134,...,The Avett Brothers,No Hard Feelings,0.983793,Nathaniel Rateliff,Redemption,0.982030,perfect,郭顶,108123,0


0    Friends told her she was better off at the bot...
1    Well I heard it, playing soft From a drunken b...
2    [Verse 1: Bill] Yeah You don't got bars that d...
3    [Verse 1] As I walk through the valley where I...
4    [Intro] Everybody shut up! (Woo!) Everyone lis...
Name: text, dtype: object

In [9]:
# Zero-shot results dataframe
df_zero_shot = pd.read_csv(path_df_zero, nrows=NROWS)
display(df_zero_shot.head())


,original_index,explicit_score
0,83131,0.999403
1,46547,0.996371
2,292,0.999687
3,22251,0.015336
4,81674,0.607372


In [10]:
# df prompting results
df_prompting = pd.read_csv(prompting_path, nrows=NROWS)
display(df_prompting.head())

,original_index,explicit_score
0,83131,1
1,46547,1
2,292,1
3,22251,0
4,81674,0


In [11]:
df_metadata_tfidf = pd.read_csv(tfidf_path, nrows=NROWS)
display(df_metadata_tfidf.head())


,test_index,predicted_label
0,83131,1
1,46547,1
2,292,1
3,22251,0
4,81674,0


In [ ]:
# 

In [12]:
# We are going to check the indexes to make sure everything aligns
if df_zero_shot['original_index'].equals(df_metadata_tfidf['test_index']):
    
    print("Indexes match between zero-shot and TF-IDF results.")
    if df_zero_shot['original_index'].equals(df_prompting['original_index']):
        print("Indexes are aligned among all dataframes.")
    else:
        print("Indexes do not match between zero-shot and prompting results. Please check.")
else:
    print("Indexes do not match between zero-shot and TF-IDF results. Please check.")



Indexes match between zero-shot and TF-IDF results.
Indexes are aligned among all dataframes.


In [13]:
df_zero_shot['thresholded_05'] = (df_zero_shot['explicit_score'] >= 0.5).astype(int)

display(df_zero_shot.head())

,original_index,explicit_score,thresholded_05
0,83131,0.999403,1
1,46547,0.996371,1
2,292,0.999687,1
3,22251,0.015336,0
4,81674,0.607372,1


In [26]:
# Wrapper for better text display
random.seed(42)
wrapper = textwrap.TextWrapper(width=100)

# Indexes in the original dataframe
indexes = df_zero_shot.loc[
    df_zero_shot['thresholded_05'] != df_metadata_tfidf['predicted_label'],
    'original_index'
].tolist()

# df_indexes = [270]
# print(f"Number of mismatched predictions: {len(indexes)}")
k = 5
random_indexes = random.sample(indexes, min(k, len(indexes)))
random_indexes.insert(0, int(df_zero_shot['original_index'][7]))
random_indexes.insert(0, int(df_zero_shot['original_index'][270]))

# print(random_indexes)
i =0
# [80549, 45621, 9452, 52992, 2282, 53349, 98388]
for idx in random_indexes: 
    # print(f"\nIndex: {idx}")
    song_name = song_names.iloc[idx]
    song_text = df_song.iloc[idx]
    artist = artists.iloc[idx]
    wrapped_text = wrapper.fill(song_text)  # Dividir en líneas de ancho 100
    # print(f"-> Song Name:{song_name}; Artist: {artist}; len:{len(song_text.split(" "))}")
    print(f"Song Text {i+1}:\n{wrapped_text}\n")

    

    # print(f"True Label: {y.iloc[idx]}")
    i += 1
    idx_in_zero_shot = df_zero_shot[df_zero_shot['original_index'] == idx].index[0]
    idx_in_tfidf = df_metadata_tfidf[df_metadata_tfidf['test_index'] == idx].index[0]
    idx_in_prompting = df_prompting[df_prompting['original_index'] == idx].index[0]
    print(f"Zero-shot Prediction: {df_zero_shot.iloc[idx_in_zero_shot]['thresholded_05']} (Score: {df_zero_shot.iloc[idx_in_zero_shot]['explicit_score']})")
    print(f"TF-IDF Prediction: {df_metadata_tfidf.iloc[idx_in_tfidf]['predicted_label']}")
    print(f"Prompting Prediction: {df_prompting.iloc[idx_in_prompting]['explicit_score']}")

Song Text 1:
[Intro] Drum Dummie Man,  I feel like that too, like (DJ Swift on the track) I  could watch the
fuckin' sun come up, you know? Never sleep, like a lil' kid (Mook got the keys jumpin')  [Chorus]
As  nights turn into days I  wonder why things won't change, yeah (Won't change, won't change) The
more I stay, the more it hurts Before  it gets better, it's gon' get worse Through all of the issues
and all of the pain I find happiness again Through all of the issues and all of the pain They  say
sun shines after it rains So as nights turn into days I wanna fly away, uh, yeah  [Verse 1] While in
my Sprinter van, I'm starin' at the cars passin' Wonderin' if it's enough space for me in thug's
mansion Ayy, fuck a rap nigga, fuck a feature, feel my life was harder I went and got it out the
mud, that's on my mama's daughters Don't get along with none of these niggas, get that from my
father You know your love make my heart race like a Daytona Charger I pour up pints of lean, I'm
tryna s